# QRACON- BB84 with Qiskit
這是一篇關於如何使用Qiskit建立一個BB84演算法並得出金鑰的筆記

希望本程式能夠在讀者理解此演算法的過程中起到些幫助，或是讓讀者在復刻此程式時能帶給讀者更多的樂趣。

---
前情提要:本程式使用Jupyter notebook和基於Qiskit = **1.1.1**編寫，如果後續存在Qiskit版本更新程式可能無法運行!
!

---
先匯入本程式所會用到的庫

In [2]:
from qiskit import QuantumCircuit
from numpy.random import randint
import numpy as np
from qiskit.primitives import StatevectorSampler

BB84的協議核心是Alice在0和1中隨機選擇並隨機用+和X兩種偏振片發送光子給Bob，然後Bob在隨機用+和X兩種偏振片測量光子。

所以我們要製作3組隨機的List來代表以上三個選擇。

In [3]:
num_qubits = 20 #可根據電腦效能更改

alice_basis = np.random.randint(2, size=num_qubits)
alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)

print(f"Alice's State:\t {np.array2string(alice_state)}")
print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

Alice's State:	 [1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 0 1]
Alice's Bases:	 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
Bob's Bases:	 [1 0 1 0 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0]


此時你可能得到像這樣的結果

Alice's State:	 [1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 1]
Alice's Bases:	 [0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1]
Bob's Bases:	 [0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0]

由上至些分別是Alice選擇的0和1、Alice選擇的偏振片(假設0代表+)、Bob選擇的偏振片。

接下來便開始製作電路。

1.根據先前的qubit數量宣告一個QuantumCircuit，使每個qubit都處於預設值$|0\rangle$。
2.在Alice選擇1的位置施加z gate使其變成$|1\rangle$。
3.在Alice選擇偏振片X的位置施加hadamard gate使其變成 $\frac{1}{\sqrt{2}} |0\rangle+\frac{1}{\sqrt{2}} |1\rangle$。
4.Bob收到qubits後對Bob選擇偏振片X的位置施加hadamard gate。
5.測量全部qubits。

In [4]:
def bb84_circuit(state, basis, measurement_basis):
    num_qubits = len(state)
    
    circuit = QuantumCircuit(num_qubits)

    for i in range(len(basis)):
        if state[i] == 1:
            circuit.x(i)
        if basis[i] == 1:
            circuit.h(i)
   
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:
            circuit.h(i)

    circuit.measure_all()
    
    return circuit

接著便可以執行電路了，本程式使用**StatevectorSampler**進行量子電腦的模擬

如果Alice與Bob使用的偏振片相同，那根據hadamard gate是酉矩陣的的性質，Bob測量的結果會與當初Alice選擇的相同，所以可做金鑰使用。
如果偏振片不同，那Bob會有50%的機率得到與Alice的選擇不同的結果。

In [5]:
qc = bb84_circuit(alice_state, alice_basis, bob_basis)
sampler = StatevectorSampler()
pub = qc
job_sampler = sampler.run([pub],shots=1).result()
result = job_sampler[0].data.meas.get_counts()

key_list = list(result.keys())[0]

encryption_key = ''
for i in range(num_qubits):
    if alice_basis[i] == bob_basis[i]:
            encryption_key += str(key_list[i])
print(f"Key: {encryption_key}")

Key: 0100010110


最後應該會長這樣 Key: 1000010110

可以發現金鑰長度大約是qubits的一半，這與我們所知道的Alice與Bob有50%會使用到相同的偏振片結果相同。

參考資料

[> The BB84 Quantum Cryptography Algorithm](https://github.com/qmunitytech/Tutorials/blob/main/intermediate/The%20BB84%20Quantum%20Cryptography%20algorithm.ipynb)

[> IBM Quantum Challenges 2024](https://github.com/qmunitytech/Tutorials/blob/main/intermediate/The%20BB84%20Quantum%20Cryptography%20algorithm.ipynb)